# ブースティング（Boosting）
逐次的にモデルを学習し、前のモデルが間違えたサンプルに対して重みを大きくすることで、全体の予測性能を向上させる手法  
弱学習器（弱い予測器）を組み合わせることで、強力な予測器を構築できる  
適切な正則化やパラメータ調整により、過学習を防ぎながら高い精度が得られる

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成： y = 2*x + 1 にノイズを加える
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.1, X.shape)

# NumPy → Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size=16):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# シンプルな学習ループ（エポック数は100）
def train_model(model, optimizer, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_tensor)
        loss = criterion(preds, y_tensor)
        loss.backward()
        optimizer.step()
    return loss.item()


In [4]:
# 1台目のモデル（初期予測器）
model1 = SimpleMLP(hidden_size=16)
optimizer1 = optim.Adam(model1.parameters(), lr=0.01)
train_model(model1, optimizer1, epochs=100)

# 残差計算： y - model1(x)
with torch.no_grad():
    residual = y_tensor - model1(X_tensor)

# 2台目のモデル（残差予測器）
model2 = SimpleMLP(hidden_size=16)
optimizer2 = optim.Adam(model2.parameters(), lr=0.01)
# 残差を目的として学習
def train_model_on_residual(model, optimizer, target, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_tensor)
        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
    return loss.item()

train_model_on_residual(model2, optimizer2, residual, epochs=100)

# ブースティングの最終予測（学習率 eta を調整）
eta = 0.5
with torch.no_grad():
    boost_preds = model1(X_tensor) + eta * model2(X_tensor)

print("Boosting ensemble final loss:", nn.MSELoss()(boost_preds, y_tensor).item())


Boosting ensemble final loss: 0.009433283470571041
